In [2]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')

# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

In [3]:
from imbdpack import skew_pro,lr_rmse_ave,lr_rmse_ave_fea,laso_rmse_ave,ElasticNet_rmse_ave,xgb_ave,svr_rmse_ave,rand_ave
from imbdpack import voting_ave,stack_ave,ave

In [4]:
fea_number=6
data=pd.read_csv('./BayesianRidge_Pre_0/Input_C_013.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
y=data.Predict
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_C_026_Y', 'Input_C_028_Y', 'Input_C_016_Y', 'Input_C_030_Y',
       'Input_C_029_Y', 'Input_C_135'], dtype=object)

## Linear Regression Baseline

In [6]:
lr_rmse_ave_fea(skew_data,6)

train_rmse: 0.0006098727775124768
test_rmse: 0.0006382861840337134
test_r2: 0.04812972267367395


## Lasso

In [5]:
laso_rmse_ave(skew_data[title],y,9e-06)

test_rmse_ave: 0.0006283247368746782
[0.0005213062870335975, 0.0005158321773938128, 0.0006580394945820789, 0.0006723433052771762, 0.0005868094010316847, 0.0006074080626218921, 0.0007063666851048812, 0.000632535794402741, 0.0007466082714303213, 0.0006359978898685955]


test_r2_ave: 0.04620633182285675
[-0.06227465186576109, 0.13938832978730786, 0.024792306630814553, -0.01902875068138843, 0.0737695788612639, 0.1047171130704384, 0.03669120498545686, 0.05792117208379621, 0.057565610753029905, 0.04852140460360932]


## ElasticNet

In [7]:
ElasticNet_rmse_ave(skew_data[title],y,[0.005],0.01)

test_rmse_ave: 0.0006307443069236679
[0.0005111529665969808, 0.0005288558135568925, 0.0006615287833108095, 0.000664486562503285, 0.0005900330978057142, 0.0006159968989378111, 0.000712282227842074, 0.0006336057082614228, 0.0007545955037459572, 0.0006349055066757318]


test_r2_ave: 0.04002400642713677
[-0.021298430539504754, 0.09538259988570341, 0.014422707379912025, 0.004648045876172291, 0.06356494461804874, 0.07921925249528061, 0.020488980873268114, 0.05473148494021429, 0.03729338213186051, 0.051787096610412475]


## SVR

In [8]:
model=SVR(kernel='rbf', C=1.3, gamma= 5e-06, epsilon= 0)
svr_rmse_ave(skew_data,6,model)

train_rmse: 0.0006377725902890571
test_rmse: 0.0006551530212427251
test_r2: -0.00046410918074893945


## XGB

In [9]:
best_xgb_model =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.15,
              booster = 'gbtree', 
              n_estimators = 110, 
              max_depth = 5, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0.05,
              subsample = 0.6,
              colsample_bytree = 0.2,
              reg_alpha =  0,
              reg_lambda = 1)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.000648536563902077
[0.00050899781611256, 0.0005562524087282393, 0.000677831078391039, 0.0006661289796088438, 0.000611026549346121, 0.0006491606772872192, 0.0007330537271913988, 0.0006563156746547682, 0.0007744748995162602, 0.0006521238281843198]


test_r2_ave: -0.014151297207583392
[-0.012704479786386447, -0.0007697756047722049, -0.034751710565168015, -0.0002784749039481671, -0.004257484130929523, -0.022594818491489033, -0.03747280553331267, -0.014244317854156696, -0.014098697613002198, -0.0003404075926689565]


## RandomForest

In [10]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 40,
            max_depth = 8,
            min_samples_split = 4,
            max_leaf_nodes = 12,
            min_samples_leaf =9,
            random_state = 42) 
rand_ave(skew_x[title], y,rand_model)

test_rmse_ave: 0.0006315265140977987
[0.0005348660605466195, 0.0005325165790511662, 0.0006659951840426542, 0.0006850500484906673, 0.0005842737222076515, 0.0006124468390785257, 0.0006928471471849549, 0.0006410886422769176, 0.0007312878383394054, 0.0006348930797594243]


test_r2_ave: 0.03324337626345417
[-0.1182553256749641, 0.08281564466215818, 0.001069264434772621, -0.05791036065784527, 0.08175700547829978, 0.08980179752816608, 0.07321291024571974, 0.03227224748367297, 0.09584636441145022, 0.051824214723111495]


In [11]:
lr = LinearRegression()
lasso_mod=Lasso(alpha=9e-06)
elastic_mod=ElasticNet(alpha=[0.005], l1_ratio=0.01)
svr_mod=SVR(kernel='rbf', C=1.3, gamma= 5e-06, epsilon= 0)

xgb_mod =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.15,
              booster = 'gbtree', 
              n_estimators = 110, 
              max_depth = 5, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0.05,
              subsample = 0.6,
              colsample_bytree = 0.2,
              reg_alpha =  0,
              reg_lambda = 1)

random_mod = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 40,
            max_depth = 8,
            min_samples_split = 4,
            max_leaf_nodes = 12,
            min_samples_leaf =9,
            random_state = 42) 
vote_mod = VotingRegressor([ ('Lasso', lasso_mod), ('SVR', svr_mod),('Elastic', elastic_mod), ('Linear', lr),
                            ('XGBRegressor', xgb_mod),('RandomForest', random_mod)])
# 
# vote_mod = VotingRegressor([ ('SVR', svr_mod), ('Linear', lr)
#                             ,('RandomForest', random_mod)])
voting_ave(skew_data[title], y,vote_mod)


test_rmse_ave: 0.0006298440141682478
[0.0005062466970852624, 0.0005248935042079542, 0.000665743187752498, 0.000666625067987765, 0.0005860252383116614, 0.0006205453141265804, 0.0007101106378388198, 0.000634314579056733, 0.0007491989907182206, 0.0006347369245969834]


test_r2_ave: 0.04313431264041252
[-0.001786785609736663, 0.1088870236365771, 0.001825063341058586, -0.0017689109019909388, 0.07624339769154875, 0.06557127723909462, 0.026452498514191625, 0.05261519062519049, 0.05101379724881838, 0.05229057461937325]


In [12]:

stack_mod = StackingRegressor(regressors=[vote_mod, rand_model,elastic_mod], 
                           meta_regressor=lasso_mod, use_features_in_secondary=True
                          )

stack_ave(skew_data[title], y,stack_mod)

test_rmse_ave: 0.0006283247368746782
[0.0005213062870335975, 0.0005158321773938128, 0.0006580394945820789, 0.0006723433052771762, 0.0005868094010316847, 0.0006074080626218921, 0.0007063666851048812, 0.000632535794402741, 0.0007466082714303213, 0.0006359978898685955]


test_r2_ave: 0.04620633182285675
[-0.06227465186576109, 0.13938832978730786, 0.024792306630814553, -0.01902875068138843, 0.0737695788612639, 0.1047171130704384, 0.03669120498545686, 0.05792117208379621, 0.057565610753029905, 0.04852140460360932]


In [13]:
ave(skew_data[title],y,0.4,stack_mod,0.4,vote_mod,0.2,lasso_mod)

test_rmse_ave: 0.0006280350082554528
[0.0005140576842811005, 0.0005184899000541492, 0.0006599054751802439, 0.0006691261798928628, 0.0005855037458636132, 0.0006117547848048273, 0.0007071809487486075, 0.0006326035248729729, 0.0007470296616323882, 0.0006346981772237627]


test_r2_ave: 0.04783526065445426
[-0.03293883126484287, 0.13049722301813338, 0.019253737266318405, -0.009300090069305433, 0.07788673220499687, 0.09185765160084125, 0.034469017991216266, 0.057719410037279095, 0.05650147925344606, 0.05240627650645957]
